In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt

from example_models import get_upper_glycolysis
from mxlpy import plot

# Metabolic control analysis

Metabolic control analysis answers the question: **what happens to the concentrations and fluxes if I slightly perturb the system?**  
It is thus a *local* measurement about which reactions hold the most control.  

The most common measurements are **elasticities** and **response coefficients**.  
The main difference between them is that *response coefficients* are calculated at **steady-state**, while *elasticities* can be calculated at every arbitrary state.


<div>
    <img src="assets/parameter-elasticity.png" 
         style="vertical-align:middle; max-height: 175px;" />
    <img src="assets/response-coefficient.png" 
         style="vertical-align:middle; max-height: 175px;" />
</div>

All the required functionality can be found in the `mxlpy.mca` module.

As an example, we will use the upper glycolysis model from the [Systems Biology textbook by Klipp et al. (2005)](https://www.wiley-vch.de/de/fachgebiete/naturwissenschaften/systems-biology-978-3-527-33636-4).

In [ ]:
from mxlpy import mca

### Variable elasticities

Variable elasticities are the sensitivity of reactions to a small change in the concentration of a variable.  
They are **not** a steady-state measurement and can be calculated for any arbitrary state.  

<img src="assets/variable-elasticity.png" style="max-width: 500px" />

Both the `concs` and `variables` arguments are optional.  
If `concs` is not supplied, the routine will use the initial conditions from the model.  
If `variables` is not supplied, the elasticities will be calculated for all variables.  

In [ ]:
elas = mca.variable_elasticities(
    get_upper_glycolysis(),
    to_scan=["GLC", "F6P"],
    variables={
        "GLC": 0.3,
        "G6P": 0.4,
        "F6P": 0.5,
        "FBP": 0.6,
        "ATP": 0.4,
        "ADP": 0.6,
    },
)

_ = plot.heatmap(elas)
plt.show()

### Parameter elasticities

Similarly, parameter elasticities are the sensitivity of reactions to a small change in the concentration of a variable.  
They are **not** a steady-state measurement and can be calculated for any arbitrary state.  

<img src="assets/parameter-elasticity.png" style="max-width: 500px" />

Both the `concs` and `parameters` arguments are optional.  
If `concs` is not supplied, the routine will use the initial conditions from the model.  
If `parameters` is not supplied, the elasticities will be calculated for all parameters.  

In [ ]:
elas = mca.parameter_elasticities(
    get_upper_glycolysis(),
    variables={
        "GLC": 0.3,
        "G6P": 0.4,
        "F6P": 0.5,
        "FBP": 0.6,
        "ATP": 0.4,
        "ADP": 0.6,
    },
    to_scan=["k1", "k2"],
)

_ = plot.heatmap(elas)
plt.show()

### Response coefficients

Response coefficients show the sensitivity of variables and reactions **at steady-state** to a small change in a parameter.  


<img src="assets/response-coefficient.png" style="max-width: 500px" />

If the parameter is proportional to the rate, they are also called **control coefficients**.

Calculation of these is run in parallel by default.  

In [ ]:
crcs, frcs = mca.response_coefficients(
    get_upper_glycolysis(),
    to_scan=["k1", "k2", "k3", "k4", "k5", "k6", "k7"],
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
_ = plot.heatmap(crcs, ax=ax1)
_ = plot.heatmap(frcs, ax=ax2)
plt.show()

<div style="color: #ffffff; background-color: #04AA6D; padding: 3rem 1rem 3rem 1rem; box-sizing: border-box">
    <h2>First finish line</h2>
    With that you now know most of what you will need from a day-to-day basis about metabolic control analysis in mxlpy.
    <br />
    <br />
    Congratulations!
</div>

## Advanced concepts / customisation

### Normalisation

By default the elasticities and response coefficients are **normalised**, so e.g. for the response coefficients the following is calculated:

$$C^J_{v_i} = \left( \frac{dJ}{dp} \frac{p}{J} \right) \bigg/ \left( \frac{\partial v_i}{\partial  p}\frac{p}{v_i} \right)$$

You can also obtain the **non-normalised** coefficients, by setting `normalized=False`, which amounts to the following calculation:

$$C^J_{v_i} = \left( \frac{dJ}{dp} \frac{p}{J} \right)$$

In [ ]:
_ = mca.response_coefficients(
    get_upper_glycolysis(),
    to_scan=["k1", "k2", "k3", "k4", "k5", "k6", "k7"],
    normalized=False,
)

### Displacement

We wrote above that we *slightly* change the value, but by how much exactly?  
By default the relative change is set to `1e-4`.  
`mxlpy` uses a **central finite difference** approximation, which means that in this case change the value to 
$$\textrm{value} \cdot \left(1 \pm 10^{-4} \right)$$

which amounts to a change of `0.01 %`. 

You can set that using the `displacement` argument.  

In [ ]:
_ = mca.response_coefficients(
    get_upper_glycolysis(),
    to_scan=["k1", "k2", "k3", "k4", "k5", "k6", "k7"],
    displacement=1e-2,
)